In [1]:
import pandas as pd
import numpy as np

In [5]:
df_stacked = pd.DataFrame()

In [2]:
nn_3_layer_v1_stacked = pd.read_csv('./3_layer_v1/result/stacked_preds.csv')

In [3]:
nn_3_layer_v1_stacked.shape

(188318, 2)

In [4]:
nn_3_layer_v1_stacked.head()

,id,3_layer_v1
0,1,1880.244751
1,2,1478.206909
2,5,4049.240479
3,10,961.461609
4,11,3311.209717


In [14]:
nn_4_layer_v1_stacked = pd.read_csv('./4_layer_v1/result/stacked_preds.csv')
print(nn_4_layer_v1_stacked.shape)
nn_4_layer_v1_stacked.head()

(188318, 2)


,id,4_layer_1
0,1,1777.215576
1,2,1415.820068
2,5,3789.947998
3,10,973.076660
4,11,3240.973145


In [10]:
xgb_v2_stacked = pd.read_csv('xgb_v2/result/stacked_preds.csv')
print(xgb_v2_stacked.shape)
xgb_v2_stacked.head()

(188318, 1)


,xgb_param#1_best_num_round
0,2020.957520
1,1762.734131
2,4593.607422
3,1008.679077
4,3080.286133


In [11]:
xgb_v3_stacked = pd.read_csv('xgb_v3/result/stacked_preds.csv')
print(xgb_v3_stacked.shape)
xgb_v3_stacked.head()

(188318, 1)


,xgb_param1_different_seed
0,2008.438232
1,1785.353149
2,4533.298340
3,991.094849
4,3136.989746


In [12]:
train = pd.read_csv('./data/train.csv')

In [13]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [19]:
y = train['loss']
print(y.shape)

(188318,)


In [15]:
df_stacked['3_layer_v1'] = nn_3_layer_v1_stacked['3_layer_v1']
df_stacked['4_layer_v1'] = nn_4_layer_v1_stacked['4_layer_1']
df_stacked['xgb_v2'] = xgb_v2_stacked['xgb_param#1_best_num_round']
df_stacked['xgb_v3'] = xgb_v3_stacked['xgb_param1_different_seed']
df_stacked.head()

,3_layer_v1,4_layer_v1,xgb_v2,xgb_v3
0,1880.244751,1777.215576,2020.957520,2008.438232
1,1478.206909,1415.820068,1762.734131,1785.353149
2,4049.240479,3789.947998,4593.607422,4533.298340
3,961.461609,973.076660,1008.679077,991.094849
4,3311.209717,3240.973145,3080.286133,3136.989746


In [17]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [24]:
linear.fit(X_train, y_train)

/Users/fujiki/anaconda3/envs/DNN/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [26]:
print(linear.coef_)
print(linear.intercept_)

[ 0.14501773  0.103913    0.79376149  0.03502063]
119.549709871


In [27]:
y_pred = linear.predict(X_test)

In [30]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

1159.652238401949

## Random Forest Regressor

In [31]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

In [32]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [33]:
y_pred = rfr.predict(X_test)

In [34]:
mean_absolute_error(y_test, y_pred)

1264.3266148470689

In [18]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [21]:
X = df_stacked.values

In [ ]:
for train_index, test_index in kf.split(X):
    